# Example of using Vue-Petite with FastHTML

Boilerplate FastHTML app (thanks Isaac for the code!)

In [1]:
from fasthtml.common import *
from fasthtml.jupyter import *
from IPython.display import display, HTML
from fasthtml.common import *
from fasthtml.jupyter import *

def create_server(app,rt):
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if 'server' in globals():
                print(f"Server already running on port {globals()['server'].port} - stopping it")
                globals()['server'].stop()
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port)
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show

Preload Vue-Petite module instance via link tag, and run fasthtml app in jupyter instance

In [2]:
vue_src = Link(href="https://unpkg.com/petite-vue?module", rel="modulepreload")

app, rt = fast_app(hdrs=[vue_src])
app, rt, server, HShow, Show = create_server(app, rt)

Starter server on port 8000


Define VueButton method, utilises dynamic module import & mounts to itself upon rendering into the DOM. 

> Fully containerising an *island* component means it keeps all state & functionality within itself. Seems to work well with HTMX swapping events in initial tests

In [3]:
def VueButton():
    id = unqid()
    script=Script("""
    import('https://unpkg.com/petite-vue?module').then(m => {
    m.createApp().mount('#%s');
    });
    """ % id)
    return Div(Button("Count: {{ count }}", **{":class": "{outline: count > 1}"}, **{"@vue:mounted": "$el.style.visibility = 'visible'; console.timeEnd('mount')"}, style="visibility:hidden", _at_click="count++", v_scope="{count: 0}", id=id), script)

@rt
def swap():
    return VueButton()

Show(Button("Swap me", hx_get="/swap", hx_swap="outerHTML", hx_target="this", onclick="console.time('mount')"))

Time taken, between pressing the swap button and the Vue component being mounted varies between the range:

> 7.3 - 23 ms

